In [1]:
# 오라클DB 접속
import oml
import oracledb
oml.core.methods.__embed__ = False

connection = oracledb.connect(user="labadmin", password="labadmin", dsn="dbserver26ai:1521/freepdb1")
cursor = connection.cursor()
#oml.connect(user="labadmin", password="labadmin", dsn="dbserver26ai:1521/freepdb1")
#cursor = oml.cursor()
# 버전 확인
dbversion = """select product,VERSION_FULL from product_component_version"""
result=cursor.execute(dbversion).fetchall()
print('DB version : ',result)

/opt/jupyterhub/vector_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DB version :  [('Oracle AI Database 26ai Free', '23.26.0.0.0')]


In [2]:
# import oml libraries
from oml.utils import ONNXPipeline,ONNXPipelineConfig

# 선구성 학습모델 조회
ONNXPipelineConfig.show_preconfigured()

['sentence-transformers/all-mpnet-base-v2',
 'sentence-transformers/all-MiniLM-L6-v2',
 'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
 'sentence-transformers/distiluse-base-multilingual-cased-v2',
 'sentence-transformers/all-MiniLM-L12-v2',
 'BAAI/bge-small-en-v1.5',
 'BAAI/bge-base-en-v1.5',
 'taylorAI/bge-micro-v2',
 'intfloat/e5-small-v2',
 'intfloat/e5-base-v2',
 'thenlper/gte-base',
 'thenlper/gte-small',
 'TaylorAI/gte-tiny',
 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
 'intfloat/multilingual-e5-base',
 'intfloat/multilingual-e5-small',
 'sentence-transformers/stsb-xlm-r-multilingual',
 'Snowflake/snowflake-arctic-embed-xs',
 'Snowflake/snowflake-arctic-embed-s',
 'Snowflake/snowflake-arctic-embed-m',
 'mixedbread-ai/mxbai-embed-large-v1',
 'openai/clip-vit-large-patch14',
 'google/vit-base-patch16-224',
 'microsoft/resnet-18',
 'microsoft/resnet-50',
 'WinKawaks/vit-tiny-patch16-224',
 'Falconsai/nsfw_image_detection',
 'WinKawaks/vit-small-patch16-224',

In [ ]:
# 제공되는 템플릿 조회
ONNXPipelineConfig.show_templates()


In [3]:
# DB관리중인 ONNX 모델 조회

managed_models = 'SELECT MODEL_NAME, MINING_FUNCTION, ALGORITHM, ALGORITHM_TYPE, (MODEL_SIZE/1024/1024) MB FROM user_mining_models ORDER BY MODEL_NAME'
models = cursor.execute(managed_models).fetchall()
print(models)

[('ALL_MINILM_L6_V2', 'EMBEDDING', 'ONNX', 'NATIVE', 86.43770408630371), ('CLIP_VIT_LARGE_PATCH14_IMG', 'EMBEDDING', 'ONNX', 'NATIVE', 292.1818504333496), ('CLIP_VIT_LARGE_PATCH14_TXT', 'EMBEDDING', 'ONNX', 'NATIVE', 119.93346118927002), ('MULTILINGUAL_E5_SMALL', 'EMBEDDING', 'ONNX', 'NATIVE', 117.3783826828003), ('VIT_BASE_PATCH16_224', 'EMBEDDING', 'ONNX', 'NATIVE', 327.6180763244629)]


In [ ]:
# 데모 초기화(ONNX 모델 삭제)
cursor.execute("BEGIN DBMS_VECTOR.DROP_ONNX_MODEL(model_name => 'VIT_BASE_PATCH16_224', force => true); END;")
print("Model removed")
managed_models = cursor.execute('SELECT MODEL_NAME, MINING_FUNCTION, ALGORITHM, ALGORITHM_TYPE, (MODEL_SIZE/1024/1024) MB FROM user_mining_models ORDER BY MODEL_NAME').fetchall()
display(managed_models)

In [ ]:
# pretrained & preconfig ONNX 모델 다운로드 
config   = ONNXPipelineConfig.from_template("text", max_seq_length=256,
           distance_metrics=["COSINE"], quantize_model=True)
pipeline = ONNXPipeline(model_name="intfloat/multilingual-e5-small",config=config)
pipeline.export2file("multilingual_e5_small", output_dir=".")
print("complete export2file")


In [ ]:
# Direct loading a Text embedding ONNX 모델 - intfloat/multilingual-e5-small
pipeline = ONNXPipeline(model_name="intfloat/multilingual-e5-small")
pipeline.export2db("multilingual_e5_small")
print("Loaded complete the model")

In [ ]:
# Direct loading a Text embedding ONNX 모델
pipeline = ONNXPipeline(model_name="sentence-transformers/all-MiniLM-L6-v2")
pipeline.export2db("all_MiniLM_L6_v2")
print("Loaded complete the model")

In [ ]:
# Direct loading a image ONNX 모델 - google/vit-base-patch16-224
pipeline = ONNXPipeline(model_name="google/vit-base-patch16-224")
pipeline.export2db("vit_base_patch16_224")
print("Loaded complete the model")

In [ ]:
# Direct loading a image ONNX 모델 - openai/clip-vit-large-patch14
pipeline = ONNXPipeline(model_name="openai/clip-vit-large-patch14")
pipeline.export2db("clip_vit_large_patch14")
print("Loaded complete the model")

In [ ]:
# DB관리중인 ONNX 모델 조회

managed_models = cursor.execute('SELECT MODEL_NAME, MINING_FUNCTION, ALGORITHM, ALGORITHM_TYPE, (MODEL_SIZE/1024/1024) MB FROM user_mining_models ORDER BY MODEL_NAME').fetchall()
display('In DB ONNX models',managed_models)